In [1]:
from models.model_teacher_vgg import CSRNet as CSRNet_teacher
model = CSRNet_teacher()
import torch
model.regist_hook()
output = model(torch.randn(1, 3, 512, 512))

In [37]:
def cal_dense_fsp(features):
    fsp = []
    for groups in features:
        for i in range(len(groups)):
            for j in range(i+1, len(groups)):
                x = groups[i]
                y = groups[j]

                norm1 = nn.InstanceNorm2d(x.shape[1])
                norm2 = nn.InstanceNorm2d(y.shape[1])
                x = norm1(x)
                y = norm2(y)
                res = gram(x, y)
                fsp.append(res)
    return fsp


def gram(x, y):
    n = x.shape[0]
    c1 = x.shape[1]
    c2 = y.shape[1]
    h = x.shape[2]
    w = x.shape[3]
    x = x.view(n, c1, -1, 1)[0, :, :, 0]
    y = y.view(n, c2, -1, 1)[0, :, :, 0]
    y = y.transpose(0, 1)
    # print x.shape
    # print y.shape
    z = torch.mm(x, y) / (w*h)
    return z

def scale_process(features, scale=[3, 2, 1], ceil_mode=True):
    # process features for multi-scale dense fsp
    new_features = []
    for i in range(len(features)):
        if i >= len(scale):
            new_features.append(features[i])
            continue
        down_ratio = pow(2, scale[i])
        pool = nn.MaxPool2d(kernel_size=down_ratio, stride=down_ratio, ceil_mode=ceil_mode)
        new_features.append(pool(features[i]))
    return new_features

In [38]:
features = model.features
features.append(output)
new_features = scale_process(features)

In [46]:
new_features[5].shape

torch.Size([1, 256, 64, 64])

In [34]:
for feature in features:
    print(feature.shape)

torch.Size([1, 64, 512, 512])
torch.Size([1, 64, 256, 256])
torch.Size([1, 128, 128, 128])
torch.Size([1, 256, 64, 64])
torch.Size([1, 512, 64, 64])
torch.Size([1, 256, 64, 64])
torch.Size([1, 1, 64, 64])


In [24]:
from torch import nn
x = features[0]
y = features[1]
norm1 = nn.InstanceNorm2d(x.shape[1])
norm2 = nn.InstanceNorm2d(y.shape[1])
x = norm1(x)
y = norm2(y)
# n = x.shape[0]
# c1 = x.shape[1]
# c2 = y.shape[1]
# h = x.shape[2]
# w = x.shape[3]
# x = x.view(n, c1, -1, 1)#[0, :, :, 0]
# y = y.view(n, c2, -1, 1)#[0, :, :, 0]
# y = y.transpose(0, 1)

In [1]:
from models.model_teacher_cnntrans import vgg19_trans
from models.model_student_cnntrans import vgg19_trans_student
import torch
teacher = vgg19_trans()
teacher.regist_hook()
student = vgg19_trans_student(direct=False)
toutput, tfeatures = teacher(torch.randn(1, 3, 512, 512))
tcnn_features = teacher.distilled_features
tcnn_features.append(toutput)
soutput, sfeatures = student(torch.randn(1, 3, 512, 512))
scnn_features = student.distilled_features
scnn_features.append(soutput)


/home/lch/anaconda3/lib/python3.9/site-packages/torch/nn/functional.py:4058: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.")


AttributeError: 'MobileNetV3Small_module' object has no attribute 'transform0'